青汁
酵素
酢・黒酢
グルコサミン
プラセンタ
コラーゲン・ヒアルロン酸
乳酸菌（便秘関連）
にんにく
消臭
お茶
その他健康食品
精力剤・精力サプリ
葉酸・妊活
ダイエットサプリ
DHA・EPA
オルニチン・しじみ
キッズサプリ
筋肉サプリ・プロテイン

被引用すうもスクレイピングしてインパクトファクターを計算する。

テキストマイニングではエビデンスに基づくであろうワードを集計
政府の機構の承認。
栄養機能食品
特別用途食品
特定保健用食品：消費者庁
ロハコ

[Scientific Journal Rankings](https://www.scimagojr.com/journalrank.php)で食品分野のジャーナルのインパクトファクターを調べた。


In [169]:
# function definition.
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
# from google.colab import files
# google adsense: <script async src="//pagead2.googlesyndication.com/pagead/js/adsby.google.js"></script>
# media net: <script src="//contextual.media.net/nmedianet.js?cid="></script>
# bidvertiser: <SCRIPT data-cfasync="false" SRC="//bdv.bidvertiser.com"
# <script src="http://scripts.chitika.net/">
# <script data-cfasync="false" type='text/javascript' src="//p100124.clksite.com"


jp_aff_list = ["https://px.a8.net/", "https://track.affiliate-b.com/", "ck.jp.ap.valuecommerce.com/", \
               "http://h.accesstrade.net", "https://j-a-net.jp/", "https://hb.afl.rakuten.co.jp", \
              "https://click.linksynergy.com", "https://www.amazon.co.jp", "googleads.g.doubleclick.net", \
              "af.moshimo.com"]

us_aff_list = ["https://www.amazon.com", "revcontent.com", "https://hb.afl.rakuten.com", \
               "www.googleadservices.com", "cat.jp.as.criteo.com"]

ncbi_link = "https://www.ncbi.nlm.nih.gov/"
journal_list = ["https://ci.nii.ac.jp", "elsevier.com"]

def get_search_results_df(keyword):
  columns = ['rank','title','url', 'affiliate_url', 'paper_url']
  df = pd.DataFrame(columns=columns)
  html_doc = requests.get('https://www.google.com/search?num=10&q=' +keyword).text
  soup = BeautifulSoup(html_doc, 'html.parser') # BeautifulSoupの初期化
  tags = soup.find_all('h3',{'class':'r'})
  rank = 1
  for tag in tags:
    title = tag.text
    print (title)
    if "の画像検索結果" in title:
        print("this is a image.")
    else:    
        url = query_string_remove(tag.select("a")[0].get("href").replace("/url?q=",""))
        affiliate_url =  get_a8_links(url) # 作成したコードの追加
        paper_url = get_paper_links(url)
        se = pd.Series([rank, title, url, affiliate_url, paper_url], columns)
        df = df.append(se, ignore_index=True)
        rank += 1
  return df
 
def query_string_remove(url):
 return url[:url.find('&')]


def get_a8_links(url):
 try:   
     html_doc = requests.get(url).text
     soup = BeautifulSoup(html_doc, 'html.parser') # BeautifulSoupの初期化
     tags = soup.select("a")
     urls = ""
     for tag in tags:
       try:
         url = tag.get("href")
         bool = any([True for x in jp_aff_list if url.find(x) > -1]) or any([True for x in us_aff_list if url.find(x) > -1])
         if bool:
           urls += url + "\n"
       except Exception as e:
         continue
     return urls
 except:
        print("error")

def get_paper_links(url):
 try:   
     html_doc = requests.get(url).text
     soup = BeautifulSoup(html_doc, 'html.parser') # BeautifulSoupの初期化
     tags = soup.select("a")
     urls = ""
     for tag in tags:
        bools = []
        scores = []   
        try:
            url = tag.get("href")
            # print(url)
            # bool = url.find(ncbi_link) > -1 or any([True for x in url_list if url.find(x) > -1]) # or any([True for x in df['journal_urls'] if url.find(x) > -1])

    #         for x, y in zip(df['simple_urls'], df['journal_scores']):
    #             bools = bools.append(url.find(x) > -1)
    #             scores = scores.append(y)
    #             print(bools)
    #         scores = scores[bools.index(True)]    
    #         scores = [scores[i] for i, x in enumerate(bools) if x == True]
            bool = any([False if x=='' else True if url.find(x) > -1 else False for x in df['simple_urls']]) \
                    or url.find(ncbi_link) > -1 or any([True for x in journal_list if url.find(x) > -1])
            # print(bool)
            if bool:
                urls += url + "\n"
        except Exception as e:
            continue
     return urls
 except:
        print("error")


In [187]:
# keywords
keyword = "kefir journal"
search_results_df = get_search_results_df(keyword)
search_results_df

,rank,title,url,affiliate_url,paper_url
0,1,"Milk kefir: nutritional, microbiological and h...",https://www.cambridge.org/core/journals/nutrit...,,https://www.cambridge.org/about-us/legal-notic...
1,2,The Microbiota and Health Promoting Characteri...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,,
2,3,"Milk kefir: composition, microbial cultures, b...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,,
3,4,"Microbiological, technological and therapeutic...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,,
4,5,(PDF) Journal Dairy Research kefir 2011 - Rese...,https://www.researchgate.net/publication/25994...,,
5,6,Properties and benefits of kefir -A review,http://rdo.psu.ac.th/sjstweb/journal/37-3/37-3...,,
6,7,Biotechnological innovations in kefir producti...,https://www.emeraldinsight.com/doi/abs/10.1108...,,
7,8,Microbiota of kefir grains,https://hrcak.srce.hr/file/144662,,
8,9,"A review: chemical, microbiological and nutrit...",https://www.tandfonline.com/doi/full/10.1080/1...,,http://authorservices.taylorandfrancis.com/\nh...
9,10,Impact of Kefir Derived Lactobacillus kefiri o...,https://www.hindawi.com/journals/jir/2015/361604/,,


In [197]:
# count the numbers of journal urls of pages.
n_journal_urls = 0
for x in range(10):
    n_journal_urls += len(search_results_df.at[x,'paper_url'].split('\n'))-1
print(n_journal_urls)    

17


In [175]:
df

,journal_titles,journal_urls,journal_scores,simple_urls
0,Comprehensive Reviews in Food Science and Food...,https://onlinelibrary.wiley.com/,2.996,onlinelibrary.wiley.com
1,Annual Review of Food Science and Technology,,2.966,
2,Trends in Food Science and Technology,https://www.elsevier.com/journals/trends-in-fo...,2.344,https://www.journals.elsevier.com/trends-in-fo...
3,Advances in Nutrition,,2.196,
4,Food Hydrocolloids,https://www.elsevier.com/journals/food-hydroco...,1.991,https://www.journals.elsevier.com/food-hydroco...
5,Food Policy,https://www.elsevier.com/journals/food-policy/...,1.950,https://www.journals.elsevier.com/food-policy
6,Global Food Security,,1.809,
7,Food Chemistry,https://www.elsevier.com/journals/food-chemist...,1.793,https://www.journals.elsevier.com/food-chemistry
8,Applied and Environmental Microbiology,https://aem.asm.org/,1.684,aem.asm.org
9,Molecular Nutrition and Food Research,http://www3.interscience.wiley.com/journal/117...,1.666,www3.interscience.wiley.com


In [73]:
# colluct journal urls from SJR.
html_doc = requests.get('https://www.scimagojr.com/journalrank.php?area=1100&category=1106').text
soup = BeautifulSoup(html_doc, 'html.parser')
tags = soup.select("a")
url_list = []
journal_urls = []

# # introduction pages of each journals
# pages = [x.get("href") for x in tags if "journalsearch.php?q=" in x.get("href")]

# docs = [BeautifulSoup(requests.get("https://www.scimagojr.com/" + x).text, 'html.parser').select("a") for x in pages]
# docs

counter = 0

for i in [x.get("href") for x in tags if "journalsearch.php?q=" in x.get("href")]:
    doc = requests.get("https://www.scimagojr.com/" + i).text
    soup = BeautifulSoup(doc, 'html.parser')
    tags = soup.select("a")
    try:
        journal_url = [requests.get(x.get("href")).url for x in tags if x.string=="Homepage"]
        if journal_url==[]:
            journal_url = [""]
        journal_urls = journal_urls + journal_url
        print(counter)    
        print(journal_urls)
    except:
        journal_urls = journal_urls + journal_url
        print("connection error.")
        print(counter)    
        print(journal_urls)
    counter += 1

# print(journal_urls)    

0
['https://onlinelibrary.wiley.com/']
1
['https://onlinelibrary.wiley.com/', '']
2
['https://onlinelibrary.wiley.com/', '', 'https://www.elsevier.com/journals/trends-in-food-science-and-technology/0924-2244#description']
3
['https://onlinelibrary.wiley.com/', '', 'https://www.elsevier.com/journals/trends-in-food-science-and-technology/0924-2244#description', '']
4
['https://onlinelibrary.wiley.com/', '', 'https://www.elsevier.com/journals/trends-in-food-science-and-technology/0924-2244#description', '', 'https://www.elsevier.com/journals/food-hydrocolloids/0268-005X#description']
5
['https://onlinelibrary.wiley.com/', '', 'https://www.elsevier.com/journals/trends-in-food-science-and-technology/0924-2244#description', '', 'https://www.elsevier.com/journals/food-hydrocolloids/0268-005X#description', 'https://www.elsevier.com/journals/food-policy/0306-9192#description']
6
['https://onlinelibrary.wiley.com/', '', 'https://www.elsevier.com/journals/trends-in-food-science-and-technology/092

In [74]:
# make df of journal urls and impact factor scores.
html_doc = requests.get('https://www.scimagojr.com/journalrank.php?area=1100&category=1106').text
soup = BeautifulSoup(html_doc, 'html.parser')
journal_titles = [i.text for i in soup.find_all('a', title='view journal details')]
journal_scores = [i.text.split()[0] for i in soup.find_all('td', class_='orde')]

df = pd.DataFrame({'journal_titles': journal_titles, 'journal_urls': journal_urls, 'journal_scores': journal_scores})

# create 'simple_urls' column to find url from html files.
simple_urls = [x.split("/")[2] if not x == "" and not x.split("/")[2] == "www.elsevier.com" else "https://www.journals.elsevier.com/" + x.split("/")[4] if not x == "" else x for x in journal_urls]
# simple_urls = [x.split("/")[2] if not x == "" or not x.split("/")[2] == www.elsevier.com else x for x in journal_urls]
df['simple_urls'] = simple_urls

df

,journal_titles,journal_urls,journal_scores
0,Comprehensive Reviews in Food Science and Food...,https://onlinelibrary.wiley.com/,2.996
1,Annual Review of Food Science and Technology,,2.966
2,Trends in Food Science and Technology,https://www.elsevier.com/journals/trends-in-fo...,2.344
3,Advances in Nutrition,,2.196
4,Food Hydrocolloids,https://www.elsevier.com/journals/food-hydroco...,1.991
5,Food Policy,https://www.elsevier.com/journals/food-policy/...,1.950
6,Global Food Security,,1.809
7,Food Chemistry,https://www.elsevier.com/journals/food-chemist...,1.793
8,Applied and Environmental Microbiology,https://aem.asm.org/,1.684
9,Molecular Nutrition and Food Research,http://www3.interscience.wiley.com/journal/117...,1.666


NameError: name 'df' is not defined

In [78]:
# collect words of foods in English and Japanese.
html_doc = requests.get('https://www.eigo-love.jp/english-word-list-food/').text
soup = BeautifulSoup(html_doc, 'html.parser')

en_words = [i.string for i in soup.find_all('td', class_='column-1')]
jp_words = [i.string for i in soup.find_all('td', class_='column-2')]

columns = ['englih_words', 'japanese_wprds']
df_words = pd.DataFrame(jp_words,en_words).reset_index()
df_words.columns = columns


In [100]:
keyword = "fermentation health food"
search_results_df = get_search_results_df(keyword)
search_results_df

Fermented Foods: Top 15 Fermented Foods & Their Benefits - Dr. Axe
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Non

,rank,title,url,affiliate_url,paper_url
0,1,Fermented Foods: Top 15 Fermented Foods & Thei...,https://draxe.com/fermented-foods/,,
1,2,7 Must-Eat Fermented Foods for a Healthy Gut -...,http://www.eatingwell.com/article/281916/7-mus...,,
2,3,The health benefits of fermenting | BBC Good Food,https://www.bbcgoodfood.com/howto/guide/health...,,
3,4,Fermented foods for better gut health - Harvar...,https://www.health.harvard.edu/blog/fermented-...,https://www.amazon.com/Always-Delicious-Satisf...,
4,5,"Food Fermentation: Benefits, Safety, Food List...",https://www.healthline.com/nutrition/fermentation,,
5,6,8 Fermented Foods to Boost Digestion and Healt...,https://www.healthline.com/nutrition/8-ferment...,,
6,7,Koji: Fermentation is Foundational | Food Cult...,https://metropolisjapan.com/fermented-food/,,
7,8,Health Benefits of Fermented Foods | Wellness ...,https://wellnessmama.com/2245/fermented-food-b...,https://www.amazon.com/REDMOND-Real-Sea-Salt-U...,
8,9,(PDF) Health Benefits of Fermented Foods - Res...,https://www.researchgate.net/publication/32002...,,


In [ ]:
english_keywords = []
japanese_keywords = []
en_n_journals = []
jp_n_journals = []

english_keywords = [x + " health" for x in en_words]
japanese_keywords = [x + "　健康" for x in jp_words]


for i, j in zip(en_words, jp_words):
    english_keyword = i + " health"
    search_results_df = get_search_results_df(english_keyword)
    en_n_journal = sum([len(x.split('\n'))-1 if not x==None else 0 for x in search_results_df['paper_url']])
    en_n_journals.append(en_n_journal)
    
    japanese_keyword = j + " 健康"
    search_results_df = get_search_results_df(japanese_keyword)
    jp_n_journal = sum([len(x.split('\n'))-1 if not x==None else 0 for x in search_results_df['paper_url']])
    jp_n_journals.append(jp_n_journal)
    
    print(english_keyword, en_n_journal, japanese_keyword, jp_n_journal)
    print(en_n_journals, jp_n_journals)    
    
keywords_n_journals_df = pd.DataFrame({'english_keyword': english_keywords, 'en_n_journals': en_n_journals, \
                                       'japanese_keyword': japanese_keywords, 'jp_n_journals': jp_n_journals})


Alcohol and Health: The Good, the Bad, and the Ugly - Healthline
What are the health effects of alcohol? | Drinkaware
Alcohol and health - Wikipedia
alcohol health の画像検索結果
this is a image.
Alcohol's Effects on the Body | National Institute on Alcohol Abuse ...
Alcohol & Your Health | National Institute on Alcohol Abuse and ...
Sorting out the health effects of alcohol - Harvard Health Blog ...
Alcohol and your health: Is none better than a little? - Harvard Health ...
No alcohol safe to drink, global study confirms - BBC News
Alcohol-related health conditions | Alcohol.org.nz
｢お酒は少量なら健康に良い｣はウソだった？ | 健康 | 東洋経済オンライン ...
健康的に飲みましょう！ お酒と健康の関わりは？ - 大阪がん循環器病 ...
お酒がもたらす身体的なメリット｜人とお酒のイイ関係｜アサヒビール
酔いのメカニズム｜人とお酒のイイ関係｜アサヒビール
飲酒の基礎知識 －公益社団法人アルコール健康医学協会－
「からだにいいアルコール」健康ランキングトップ10 | Laifu
適正飲酒のすすめ | 知っておこう！上手な飲み方、付き合い方 | サッポロ ...
適量ってどのくらい？｜DRINK SMART お酒の正しい付き合い方を考え ...
アルコールと健康の関係は？ 「酒は百薬の長」は本当か | ニュース | 保健 ...
「アルコールは少量でも健康に悪い」がんのリスクの可能性も ...
alcohol health 86 アルコール飲料 健康 0
[86] [0]
4 Benefits of 

In [254]:
keywords_n_journals_df

,english_keyword,en_n_journals,japanese_keyword,jp_n_journals
0,alcohol health,87,アルコール飲料 健康,0
1,apple juice health,57,リンゴジュース 健康,0
2,beer health,0,ビール 健康,0


In [256]:
search_results_df

,rank,title,url,affiliate_url,paper_url
0,1,なぜなに？コーヒーと健康 - ネスレ日本,http://www.nestle.co.jp/nhw/coffee/naze,,
1,2,なぜなに？コーヒーと健康 - ネスレ日本,http://www.nestle.co.jp/nhw/coffee/naze,,
2,3,一日何杯までOK？ コーヒーの健康効果5つ＆賢い飲み方 - macaroni,https://macaro-ni.jp/9081,,
3,4,コーヒーの健康効果とは | 健康長寿ネット,https://www.tyojyu.or.jp/net/kenkou-tyoju/kour...,,http://onlinelibrary.wiley.com/doi/10.1111/ijd...
4,5,コーヒーと健康 ｜ 全日本コーヒー協会,http://coffee.ajca.or.jp/webmagazine/health,,
5,6,コーヒーは健康にいい。16年間、調べてわかったこと（研究結果 ...,https://www.huffingtonpost.jp/2017/07/13/coffe...,None,None
6,7,本当はどっち？コーヒーは体に良い？悪い？ - NAVER まとめ,https://matome.naver.jp/odai/2138051001421264601,,
7,8,2/2 コーヒーは身体にいい？悪い？ポリフェノールの健康効果とは | 太陽 ...,https://fufufu.rohto.co.jp/series/jyoshiki/173/2/,,
8,9,コーヒーは体に悪い？良い？健康への影響を徹底的に調べた | アルパ ...,https://www.arupakano.com/entry-2018-02-16-224251,https://www.amazon.co.jp/%E3%83%9E%E3%82%A6%E3...,https://www.ncbi.nlm.nih.gov/pubmed/22394208\n
9,10,「コーヒーは健康に良い」は本当か 何杯までなら飲んで良いのか ...,http://tokuteikenshin-hokensidou.jp/news/2018/...,,


9